---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [158]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [159]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [160]:
# n=len(P1_Graphs[4].nodes())

# inst_= [i[1] for i in list(P1_Graphs[4].degree().items())]

# degree_values = sorted(set(inst_))

# import matplotlib.pyplot as plt


# prob_dist = [sorted(inst_).count(i) / n for i in degree_values]

# plt.bar(degree_values, prob_dist)


In [161]:
def graph_identification():
    
#     Your Code Here
    
    return ['PA', 'SW_L','SW_L','PA','SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [162]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [163]:
def salary_predictions():
    
    df=pd.DataFrame(index=G.nodes())

    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    df['degree_centrality'] = pd.Series(nx.degree_centrality(G))
    df['closeness'] = pd.Series(nx.closeness_centrality(G, normalized=True))
    df['betweeness'] = pd.Series(nx.betweenness_centrality(G, normalized=True))
    df['pr'] = pd.Series(nx.pagerank(G))    
    
    df['ManagementSalary'] = nx.get_node_attributes(G, 'ManagementSalary').values()
    test = df[df['ManagementSalary'].isnull()]
    train = df.dropna()
    
    X_train = train.drop('ManagementSalary',axis=1)
    y_train = train['ManagementSalary']
    X_test = test.drop('ManagementSalary',axis=1)
    
    from sklearn.preprocessing import MinMaxScaler
    scale = MinMaxScaler()
    Xtrain_s = scale.fit_transform(X_train)
    Xtest_s = scale.transform(X_test)

    from sklearn.neural_network import MLPClassifier
    clf = MLPClassifier(hidden_layer_sizes = [10, 5], alpha = 5,
                       random_state = 0, solver='lbfgs', verbose=0)
    clf.fit(Xtrain_s, y_train)

    ans = pd.Series(clf.predict_proba(Xtest_s)[:,1], index=test.index)
    
    return ans
salary_predictions()

1       0.128439
2       0.586261
5       0.979859
8       0.133702
14      0.304625
18      0.208766
27      0.269586
30      0.336144
31      0.169191
34      0.143513
37      0.096221
40      0.195036
45      0.162267
54      0.249954
55      0.252905
60      0.176908
62      0.992842
65      0.551284
77      0.078478
79      0.132108
97      0.099812
101     0.057517
103     0.214342
108     0.122930
113     0.454489
122     0.044350
141     0.396296
142     0.863703
144     0.072049
145     0.419966
          ...   
913     0.045734
914     0.078520
915     0.002087
918     0.144304
923     0.035175
926     0.090079
931     0.055969
934     0.013933
939     0.011147
944     0.002353
945     0.039608
947     0.162715
950     0.098173
951     0.038313
953     0.027345
959     0.002922
962     0.002440
963     0.139886
968     0.110279
969     0.104672
974     0.095061
984     0.012668
987     0.110375
989     0.113364
991     0.116944
992     0.003956
994     0.003077
996     0.0024

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [165]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [197]:
def new_connections_predictions():

    index_ = [i[:2] for i in list(nx.jaccard_coefficient(G))]

    df = pd.DataFrame(index=index_)

    df['jaccard_coefficient'] = [i[2] for i in list(nx.jaccard_coefficient(G))]
    df['resource_allocation_index'] = [i[2] for i in list(nx.resource_allocation_index(G))]
    df['adamic_adar_index'] = [i[2] for i in list(nx.adamic_adar_index(G))]
    df['preferential_attachment'] = [i[2] for i in list(nx.preferential_attachment(G))]

    for node in G.nodes():
        G.node[node]['community'] = G.node[node]['Department']


    df['cn_soundarajan_hopcroft'] = [i[2] for i in list(nx.cn_soundarajan_hopcroft(G))]
    df['ra_index_soundarajan_hopcroft'] = [i[2] for i in list(nx.ra_index_soundarajan_hopcroft(G))]

    final_df = future_connections.join(df)

    test = final_df[final_df['Future Connection'].isnull()].drop('Future Connection',axis=1)

    train = final_df.dropna()

    X_train = train.drop('Future Connection', axis=1)
    y_train = train['Future Connection']

    from sklearn.preprocessing import MinMaxScaler
    scale = MinMaxScaler()
    Xtrain_s = scale.fit_transform(X_train)
    Xtest_s = scale.transform(test)

    from sklearn.neural_network import MLPClassifier
    clf = MLPClassifier(hidden_layer_sizes = [10, 5], alpha = 5,
                           random_state = 0, solver='lbfgs', verbose=0)
    clf.fit(Xtrain_s, y_train)

    ans = pd.Series(clf.predict_proba(Xtest_s)[:,1], index=test.index)
    
    return ans

new_connections_predictions()


(107, 348)    0.020368
(542, 751)    0.013103
(20, 426)     0.597908
(50, 989)     0.013099
(942, 986)    0.013094
(324, 857)    0.013099
(13, 710)     0.137553
(19, 271)     0.128170
(319, 878)    0.013097
(659, 707)    0.013103
(49, 843)     0.013096
(208, 893)    0.013102
(377, 469)    0.013302
(405, 999)    0.014508
(129, 740)    0.014282
(292, 618)    0.015654
(239, 689)    0.013097
(359, 373)    0.013216
(53, 523)     0.138420
(276, 984)    0.013096
(202, 997)    0.013095
(604, 619)    0.055641
(270, 911)    0.013097
(261, 481)    0.060750
(200, 450)    0.999994
(213, 634)    0.013105
(644, 735)    0.040110
(346, 553)    0.013116
(521, 738)    0.013139
(422, 953)    0.014892
                ...   
(672, 848)    0.013097
(28, 127)     0.943815
(202, 661)    0.013118
(54, 195)     0.999923
(295, 864)    0.013100
(814, 936)    0.013104
(839, 874)    0.013094
(139, 843)    0.013100
(461, 544)    0.013155
(68, 487)     0.013157
(622, 932)    0.013101
(504, 936)    0.014697
(479, 528) 